# Autoencoder Model

**Authors:** Jai Sharma, Christopher Sun, Milind Maiti

**Date:** 2022.06.16

**Description:** This module defines an Autoencoder Deep Learning model for the research paper titled "A Deep Learning Ensemble Framework for Off-Nadir Geocentric Pose Prediction." The tasks here include:

1. Define an encoder Deep Learning network which can map a satellite image (from the IEEE dataport) to an embedding of a predetermined size (512).
2. Define a decoder Deep Learning network which can map an embedding to an image with the same dimensions as the mentioned satellite images.
3. Create a Sequential Keras model which connects the encoder and decoder networks into one autoencoder network with a goal of recovering the original satellite image after creating an embedding.
4. Visualize the robustness of the autoencoder model, as explained in number (3).
5. Use Multidimensional Scaling (MDS) to visualize the embeddings of the encoder model.

## Import Libraries and Data

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
import random
import math
import gc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import *
from tensorflow.keras.layers import *
from sklearn.manifold import MDS
from IPython.display import FileLink

# Print Confirmation
print("Setup Complete")

Setup complete.


In [ ]:
# Import Data
imgs = pickle.load(open("/kaggle/input/geocentric-pose-analysis-of-satellite-imagery/j2k_imgs.dat", "rb"))
y = pickle.load(open("/kaggle/input/geocentric-pose-analysis-of-satellite-imagery/y.dat", "rb"))

## Define and Train Autoencoder Model

In [4]:
# CNN Autoencoder
# Encoder
dim = 512
input_img = keras.Input(shape=imgs.shape[1:])
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(x)
shape = x.shape
encoded = Dense(dim)(x)
encoder = Model(inputs=input_img, 
                outputs=encoded)

# Decoder
encoded_input_img = keras.Input(shape=(dim))
x = Dense(shape[1])(encoded_input_img)
x = Reshape((int(shape[1]**0.5/4),int(shape[1]**0.5/4),16))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='relu', padding='same')(x)
decoder = Model(inputs=encoded_input_img,
               outputs=decoded)

# AutoEncoder
autoencoder = Sequential([encoder, decoder])
autoencoder.compile(optimizer='adam', loss='mae', metrics=["mse"])

In [ ]:
# View the encoder model architecture summary
encoder.summary()

In [ ]:
# View the decoder model architecture summary
decoder.summary()

In [ ]:
# View the autoencoder model architecture summary
autoencoder.summary()

In [ ]:
# Train Autoencoder
history = autoencoder.fit(imgs, imgs, epochs=1000, batch_size=16)

In [ ]:
# Save Autoencoder weights
autoencoder.save_weights("autoencoder.h5")

In [ ]:
# Encode the first 1000 images and get the decodings
encoded_imgs = autoencoder.layers[0].predict(imgs[:1000], batch_size=16)
decoded_imgs = autoencoder.predict(imgs[:1000], batch_size=16).astype(int)

## Visualize Robustness of Autoencoder Model

In [ ]:
# Visualize robustness of Autoencoder model

n = 10  # Number of images displayed
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original image
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(imgs[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction image
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(256, 256, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Visualizing Autoencoder Embeddings using Multidimensional Scaling (MDS)

In [ ]:
# Color points based on scale
colors = y[:,0]
embedding = MDS(n_components=2)
transformed_imgs = embedding.fit_transform(encoded_imgs)
plt.scatter(transformed_imgs[:,0],transformed_imgs[:,1], c=colors[:transformed_imgs.shape[0]], cmap="viridis")
plt.show()

In [ ]:
# Color points based on angle
colors = y[:,1]
plt.scatter(transformed_imgs[:,0],transformed_imgs[:,1], c=colors[:transformed_imgs.shape[0]], cmap="viridis")
plt.show()